In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import re
import numpy as np
import pandas as pd
from helpers import preprocess_str, clean
from vectorizer import vectorize
from grid_search import grid_search
from collections import defaultdict
from fuzzywuzzy import fuzz
from collections import Counter
import cPickle

In [3]:
!awk -F"\"" '!$NF{print;next}{printf("%s ", $0)}' realty_company.csv > realty_company_no_newline.csv

with open('realty_company_no_newline.csv','rb') as f:
    super_list = []
    for i, line in enumerate(f):
        id_, ver, names_str = line.strip().split(',',2)
        super_list.append([id_, ver, clean(names_str)])

df_company = pd.DataFrame(super_list[1:], columns=super_list[0])
df_company['name_clean'] = df_company['name'].apply(preprocess_str)
df_company[['name','name_clean']][:5]

,name,name_clean
0,Cushman & Wakefield,cushman wakefield
1,"Durst Organization, Inc.",durst
2,Oxford & Simpson,oxford simpson
3,UGL Equis,ugl equis
4,Studley,studley


In [4]:
corpus=df_company['name_clean'].unique()
matrix_cv = vectorize(
    corpus=df_company['name_clean'].unique(), 
    ngrams=2, 
    tokenizer='char'
)


In [5]:
matrix_cv.shape, corpus.shape

((9473, 830), (9473,))

In [6]:
#jaccard at 0.2 did the best

metrics = [
#     'cosine',
#     'euclidean',
#     'manhattan',
#     'hamming',
#     'chebyshev',
#     'canberra',
#     'braycurtis',
    'jaccard',
#     'dice'
]

dict_of_cluster_objs = grid_search(metrics, matrix_cv, range_of_eps_distances=[0.06, 0.09, 0.2,0.4])

JACCARD

distance:  0.06  cnt:  129
-1     9177
 7       10
 11       8
 70       6
 20       6
 73       6
 16       4
 65       4
 8        3
 91       3
 37       3
 38       3
 56       3
 57       3
 69       3
...
67     2
114    2
81     2
113    2
71     2
72     2
112    2
74     2
75     2
76     2
77     2
78     2
79     2
80     2
0      2
Length: 129, dtype: int64
count clusters:  129 large clusters:  1
--------------------------------------------------

distance:  0.09  cnt:  210
-1      8962
 48       17
 17       14
 106      13
 13       11
 0        11
 64        7
 4         5
 155       4
 168       4
 36        4
 68        4
 28        4
 45        3
 154       3
...
142    2
141    2
140    2
137    2
126    2
136    2
135    2
133    2
132    2
131    2
130    2
129    2
128    2
127    2
104    2
Length: 210, dtype: int64
count clusters:  210 large clusters:  6
--------------------------------------------------

distance:  0.2  cnt:  622
-1      7555
 6       

In [14]:
dbscan, cnt= dict_of_cluster_objs['jaccard']['0.2']
best_cluster = pd.Series(dbscan.labels_)

for i in best_cluster.value_counts().index[:5]:
    if i != -1: 
        print 'CLUSTER#', i
        print corpus[best_cluster[best_cluster == i].index]


CLUSTER# 6.0
['colliers international' 'collier international'
 'colliers international sj' 'colliers intemal ional' 'colliers int'
 'colliers international san jose' 'colliers international oakland'
 'colliers international pleasanton'
 'studley colliers international oakland' 'colliers international fair'
 'colliers international sf' 'colliers international pl'
 'colliers international oak' 'colliers international wc'
 'colliers international rwc' 'colliers international cbre'
 'colliers int w' 'colliers lnternational' 'colliers seeley international'
 'colliers internatioanl' 'colliers internationa'
 'colliers international jll' 'colliers internatiional'
 'colliers international roseville' 'colliers international sf jon jensen'
 'cbre colliers international' 'colleirs international'
 'collieres international' 'colliers lntemational'
 'colliers seeley international colliers international'
 'colliers seeley international lee' 'colliers intl' 'colliers inter'
 'colliers international cr

In [18]:
reverse_dictionary_with_list_vals = lambda x: \
    {item: clust_name for clust_name, cluster in x.items() for item in cluster}

def cluster_to_hashmap(clusters):
    stor = {}
    for i in clusters.value_counts().index:
        target_cluster = corpus[clusters[clusters == i].index]
        if i != -1:
            stor[target_cluster[0]] = target_cluster

    return reverse_dictionary_with_list_vals(stor)

name_clean_map = cluster_to_hashmap(best_cluster)

df_company['name_clean_post_clustering'] = df_company['name_clean'].apply(lambda x: name_clean_map.get(x) if name_clean_map.get(x) else x)
df_company['name_clean_post_clustering']

0          cushman wakefield
1                      durst
2             oxford simpson
3                  ugl equis
4                    studley
5                grubb ellis
6     colliers international
7             vornado office
8                        jll
9           cb richard ellis
10            cassidy turley
11                  sl green
12                     cresa
13                       abs
14           prime manhattan
...
13820          monday berger
13821                       
13822       horowitz studley
13823                mandell
13824    jane roundell cresa
13825             r w holmes
13826      cushman wakefield
13827                     cw
13828                      w
13829           studley peck
13830                w cento
13831                  prime
13832    100 franklin street
13833                klemmer
13834                    fpk
Name: name_clean_post_clustering, Length: 13835, dtype: object

In [19]:
uncat = sorted(corpus[best_cluster[best_cluster == -1].index])

stor = []
for str_ in uncat:
    try:
        stor.append(str_.split()[0])
    except:
        print str_


missed_names_top_40 = pd.Series(stor).value_counts()[:40].index

def stop_word_generator(initial_stop_words, corpus):
    all_words = list(corpus)
    stor_dict = defaultdict(list)
    threshold_value = 80
    for word in initial_stop_words:
        for curr_word in all_words:
            if len(curr_word) == 1: continue
            if fuzz.token_set_ratio(word, curr_word) > threshold_value:
                stor_dict[word].append(curr_word)
    return stor_dict

temp_dic = reverse_dictionary_with_list_vals(stop_word_generator(missed_names_top_40, uncat))
df_company['name_clean_post_clustering_refined'] = df_company['name_clean_post_clustering'].apply(lambda x: temp_dic.get(x) if temp_dic.get(x) else x)
df_company['name_clean_post_clustering_refined'][:5]

0    cushman wakefield
1                durst
2       oxford simpson
3            ugl equis
4              studley
Name: name_clean_post_clustering_refined, dtype: object

In [20]:
from collections import defaultdict

def adhoc_find_similar_names(initial_stop_words, corpus):
    all_words = [word for sub_list in [item.split() for item in list(corpus)] for word in sub_list]
    stor_dict = defaultdict(list)
    threshold_value = 80
    for word in initial_stop_words:
        for curr_word in all_words:
            if fuzz.ratio(word, curr_word) > threshold_value:
                stor_dict[word].append(curr_word)
    return stor_dict.values()

map(lambda x: set(x), adhoc_find_similar_names(pd.Series(stor).value_counts()[:4].index, uncat))

[{'bre', 'cbe', 'cbre', 'cre', 'dcbre'},
 {'colleirs', 'collier', 'colliers', 'collirs', 'colllers'},
 {'nai'},
 {'jll', 'jllr', 'jlly'}]

In [21]:

df_brokers = cPickle.load(open('../df_brokers.pkl', 'rb'))

df_brokers[:5]

,id,version,name
0,1,0,david pinetta
1,2,0,abe bose
2,2,0,joel weinberg
3,3,0,john mambrino
4,4,0,luke greene


In [22]:
df_comp_master_tenant_realty_company = pd.read_csv('comp_master_tenant_realty_company.csv')
df_comp_master_tenant_realty_company[:5]

,comp_id,realty_company_id
0,3,1
1,4,564
2,11,3
3,14,1
4,16,4


In [23]:
df_comp_master_tenant_realty_broker = pd.read_csv('comp_master_tenant_realty_broker.csv')
df_comp_master_tenant_realty_broker[:5]

,comp_id,realty_broker_id
0,3,145
1,4,54
2,11,1
3,14,1784
4,18,3


In [24]:
master_table = pd.merge(df_comp_master_tenant_realty_broker, 
                        df_comp_master_tenant_realty_company,                         
                       how='left')
master_table[:5]

,comp_id,realty_broker_id,realty_company_id
0,3,145,1
1,4,54,564
2,11,1,3
3,14,1784,1
4,18,3,5


In [28]:
indices = df_company[df_company['name_clean_post_clustering_refined'] == ''].index
df_company['name_clean_post_clustering_refined'].ix[indices] = df_company['name'].ix[indices] 
df_company['name_clean_post_clustering_refined'][:5]

0    cushman wakefield
1                durst
2       oxford simpson
3            ugl equis
4              studley
Name: name_clean_post_clustering_refined, dtype: object

In [29]:
#some names got cleared out by the preprocessor, the stop_word set

df_company['name_clean_post_clustering_refined'].value_counts()

cbre                      146
colliers international    124
cushman wakefield         119
re                        113
cornish carey sm          111
w                         104
colliers                  103
newmark knight frank       99
lee                        95
cassidy turley             95
grubb                      92
jll                        86
nai                        81
pacific                    76
cushman                    74
...
griffin                          1
swsg                             1
codina                           1
heher                            1
Commercial Realty Advisors       1
lennox financial                 1
holloway                         1
ugl justin helbig greg hilton    1
Commercial Realty                1
812 lp                           1
herschel plaza                   1
catalyst                         1
freeman myre                     1
olive limited                    1
myking                           1
Length: 6934, dtype: 

In [30]:
df_brokers = df_brokers.rename(columns={'id':'realty_broker_id'})
df_company = df_company.rename(columns={'id':'realty_company_id'})
df_company['realty_company_id'] = df_company.realty_company_id.astype(int)
df_brokers['realty_broker_id'] = df_brokers.realty_broker_id.astype(int)
master_table = pd.merge(master_table, df_company, how='right', on='realty_company_id')
master_table

,comp_id,realty_broker_id,realty_company_id,version_x,name_x,name_clean_x,name_clean_post_clustering_x,name_clean_post_clustering_refined_x,version_y,name_y,name_clean_y,name_clean_post_clustering_y,name_clean_post_clustering_refined_y
0,3,145,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
1,14,1784,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
2,111,1796,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
3,136,23929,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
4,136,3971,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
5,202,13640,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
6,329,35,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
7,444,1788,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
8,462,1597,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
9,877,1791,1,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield,0,Cushman & Wakefield,cushman wakefield,cushman wakefield,cushman wakefield
